In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import trange
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from copy import deepcopy as copy
from tqdm import tqdm
import torch.optim as optim
import random
import re
import os
import jieba
def seed_torch(seed=1122):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
BERT_PATH = 'bert-base-chinese/'

In [3]:
datalist = os.listdir('comment_data')
print(len(datalist))
datalst = []
for i in datalist:
    print(i)
    try:
        if i[-1] == 'x':
            data = pd.read_excel('comment_data/'+i).drop_duplicates(subset=['店铺名称','用户名','评论'],keep='first')
        else:
            data = pd.read_csv('comment_data/'+i).drop_duplicates(subset=['店铺名称','用户名','评论'],keep='first')
        datalst.append(data)
    except:
        print(i)

69
一只酸奶牛（财大店）.xlsx
三牦记.牛肉火锅.csv
九孔莲藕汤锅（总店）.csv
乡村基（国色天香店）.csv
书亦烧仙草(温江财大二店).xlsx
五味鲜肉抄手（温江总店）.csv
伍氏猪脚（柳浪湾店）.csv
众里寻她·千百度自助烤肉（温江店）.csv
佘记全家福芋儿鸡（温江店）.xlsx
北木南和烤肉（西南财大店）.xlsx
华莱士评论采集.csv
台新石头火锅（温江店）.csv
司乎 日式鲜吐司（新光天地店）.csv
吉布鲁牛排海鲜自助（温江合生汇店）.xlsx
嘎嘎鸭脑壳（温江南浦店）.xlsx
在首尔韩国料理•自助烤肉（温江店）.csv
大美蓉城自助火锅烤肉（温江店）.csv
奇味干锅馆闷锅乡.xlsx
好利来（温江合生汇店）.csv
守柴炉烤鸭（温江新尚天地店）.csv
官人桥蒙自过桥米线.csv
宾GO101牛排披萨意面.xlsx
尼克熊西餐（财大东门店）.csv
屋头串串（临江路店）.csv
川西坝子火锅（温江2.0店）.csv
巴色鱼捞（温江直营店）.xlsx
柳飘飘亚洲烤肉集合店.xlsx
正新鸡排（繁华广场店）.xlsx
汉堡王（温江新尚天地店）.csv
江南道.xlsx
江南道自助烤肉海鲜涮锅（温江大学城店）.csv
泥巴小院市井火锅（温江店）.csv
泰澜得.csv
泰澜得.csv.csv
渝味晓宇火锅（温江店）.csv
烤匠评论采集_790.csv
烤鱼匠麻辣烤鱼.xlsx
爱达乐蛋糕.xlsx
牛蹄筋老面馆（财大店）.xlsx
独二家私房面.xlsx
猪圈小米辣（温江1店）.csv
王婆婆大牌檔.csv
码头故事火锅.csv
紫燕百味鸡.xlsx
芭夯兔.xlsx
芭莉与彩虹西餐厅（成都合生汇店）.csv
蛙三泡椒牛蛙（温江合生汇店）.xlsx
蜀大侠去重评论采集.csv
蜜语之恋.xlsx
裕昌火锅（德通桥路总店）.csv
诺曼汀艺术烘焙（财大店）.csv
谦和清心素食（温江店）.csv
豪客来牛排（成都温江新尚天地店）.csv
辛麻道火锅.xlsx
锦城大院火锅.csv
锦荷苑自助火锅.csv
院坝记忆牦牛串串·龙虾·烧烤（鱼凫路总店）.csv
雅府正红木桶鱼.csv
青椒鱼（温江大学城店）.xlsx
韩呈一品韩式自助（温江店）.xlsx
韩国小胖料理自助烤肉.csv
韩婆婆小院烧烤.xlsx
香街坊汤锅馆（国色天香店

In [5]:
for data in datalst:
    data.dropna(axis=0,subset = ["评论"]) 
pd.concat(datalst).to_csv("全部评论数据.csv",encoding="utf-8-sig")

# 测试情感分析模型是否有效

In [21]:
comment = pd.read_csv('全部评论数据.csv')
comment = comment[:100]

In [16]:
class GetBERT(nn.Module):

    def __init__(self):
        super(GetBERT, self).__init__()
        self.bert_tokenizer = BertTokenizer.from_pretrained("C:/Users/12968/Desktop/chinese-bert-wwm-ext")
        self.bert = BertModel.from_pretrained("C:/Users/12968/Desktop/chinese-bert-wwm-ext")
        for param in self.bert.parameters():
            param.requires_grad = True
        
    def forward(self, sentence_lists):
        """
        输入句子列表(去掉了停用词的)
        """
        sentence_lists = [' '.join(x) for x in sentence_lists]
        #print('sentence_lists:'+str(sentence_lists))
        ids = self.bert_tokenizer(sentence_lists, padding=True, return_tensors="pt")
        #print('ids:'+str(ids))
        inputs = ids['input_ids']
        #print('inputs:'+str(inputs))

        embeddings = self.bert(inputs)
        #print(str(embeddings[0].shape))
        x = embeddings[0] #1 * 768
        #print(x.shape)
        return x
    
class Pre:
    def __init__(self, text):
        """
        输入一个文本
        """
        self.puncs_coarse = ['。', '!', '；', '？', '……', '\n',' ']
        self.text = text
        self.stopwords = self.deal_wrap('dict/stop1205.txt')
    
    def segment(self, sentence):
        sentence_seged = jieba.cut(sentence.strip())
        outstr = ''
        for word in sentence_seged:
            if word not in self.stopwords:
                if word != '\t':
                    outstr += word
                    outstr += " "
        word_list = outstr.split(' ')
        pattern = '[A-Za-z]*[0-9]*[\'\"\%.\s\@\!\#\$\^\&\*\(\)\-\<\>\?\/\,\~\`\:\;]*[：；”“ ‘’+-——！，。？、~@#￥%……&*（）【】]*'
        t = [re.sub(pattern, "", x.strip()) for x in word_list]
        t = [x for x in t if x != '']
        return ''.join(t)
    
    def deal_wrap(self, filedict):
        temp = []
        for x in open(filedict, 'r', encoding='utf-8').readlines():
            temp.append(x.strip())
        return temp
        
    def split_sentence_coarse(self):
        """
        按照。！？“”等中文完整句子语义来分句
        1. 去除换行符、多余的空格、百分号
        2. 分句，存入列表
        :return:装着每个句子的列表（包括标点符号）
        """
        
        text = self.text
        sentences = []
        start = 0
        for i in range(len(text)):
            if text[i] in self.puncs_coarse:
                sentences.append(text[start:i + 1])
                start = i + 1
        if start == 0:
            sentences.append(text)
        return sentences
    
    def get_keywords(self, data):
        """
        如果句子太长，就进行关键词提取
        """
        from jieba import analyse
        textrank = analyse.textrank
        keywords = textrank(data, topK=8)
        return ''.join(keywords)

    def preprocess(self):
        # 分句
        sentences = self.split_sentence_coarse()
        # 对每个句子，去除里面的停用词，再连起来
        # 对每个句子，如果句子太长，长度大于20（我随便定的），就抽取八个关键词连起来
        new_sent = []
        for i in sentences:
            if len(i) < 5:
                new_sent.append(i)
                continue
            i = self.segment(i)
            if len(i) > 25:
                i = self.get_keywords(i)
            if i != '':
                new_sent.append(i)
        return new_sent
    
class GetData():
    def __init__(self,pos=4000, neg=3600):
        data = pd.read_excel('E:/FinancialIntelligence/sentiment_classify_data/comments_raw_v1.xls')
        data = data[data['score']!=3].reset_index()
        data['label'] = data['score'].map(lambda a : 1 if a in [4,5] else 0) 
        data.drop(['id','score'],inplace=True,axis=1)
        data['content'] = [str(i) for i in list(data['content'])]
        # 原数据标签为0（负向情感）的数据有3632条，正向情感的有57262条
        data1 = data[data['label']==1].sample(pos)
        data0 = data[data['label']==0].sample(neg)
        data = pd.concat([data1,data0],axis=0,ignore_index=True)
        self.data = data
    
    def split_sen(self):
        x = []
        y = []
        for i in trange(len(self.data)):
            p = Pre(self.data['content'][i])
            sen_lst = p.preprocess()
            if sen_lst == []:
                continue
            x.append(sen_lst)
            y.append(self.data['label'][i])
        print(len(x))
        print(y.count(1))
        print(y.count(0))
        return x, y

class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm_layer = nn.LSTM(input_size=768, hidden_size=128, batch_first=True)
        self.linear_layer = nn.Linear(in_features=128, out_features=2, bias=True)
        
    def forward(self, x):
        out1, (h_n, h_c) = self.lstm_layer(x)
        a, b, c = h_n.shape
        out = self.linear_layer(h_n.reshape(a*b, c))
        out = F.log_softmax(out,dim=1)
        return out
    
def train_model(epoch, train_dataLoader, test_dataLoader):
    # 训练模型
    best_model = None
    train_loss = 0
    test_loss = 0
    best_loss = 100
    epoch_cnt = 0
    for _ in range(epoch):
        total_train_loss = 0
        total_train_num = 0
        total_test_loss = 0
        total_test_num = 0
        for x, y in tqdm(train_dataLoader,
                         desc='Epoch: {}| Train Loss: {}| Test Loss: {}'.format(_, train_loss, test_loss)):
        #for x, y in train_dataLoader:
            x_num = len(x)
            p = model(x)
            loss = loss_func(p, y.long())
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            total_train_loss += loss.item()
            total_train_num += x_num
        train_loss = total_train_loss / total_train_num
        train_loss_list.append(train_loss)
        for x, y in test_dataLoader:
            x_num = len(x)
            p = model(x)
            loss = loss_func(p, y.long())
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            total_test_loss += loss.item()
            total_test_num += x_num
        test_loss = total_test_loss / total_test_num
        test_loss_list.append(test_loss)
        
        # early stop
        if best_loss > test_loss:
            best_loss = test_loss
            best_model = copy(model)
            torch.save(best_model.state_dict(), 'lstm_.pth')
            epoch_cnt = 0
        else:
            epoch_cnt += 1
            
        if epoch_cnt > early_stop:
            torch.save(best_model.state_dict(), 'lstm_.pth')
            print("保存模型")
            #print(best_model.state_dict())
            break
def test_model(test_dataLoader_):
    pred = []
    label = []
    model_.load_state_dict(torch.load("lstm_.pth"))
    model_.eval()
    total_test_loss = 0
    total_test_num = 0
    for x, y in test_dataLoader_:
        x_num = len(x)
        p = model_(x)
#         print('##', len(p), len(y))
        loss = loss_func(p, y.long())
        total_test_loss += loss.item()
        total_test_num += x_num
        pred.extend(p.data.squeeze(1).tolist())
        label.extend(y.tolist())
    test_loss = total_test_loss / total_test_num
    # print('##', len(pred), len(label))
    return pred, label, test_loss, test_loss_list


In [23]:
bert = GetBERT()
model = LSTM()
model.load_state_dict(torch.load("lstm_.pth"))
model.eval()
comment['sent'] = 99
for i in trange(len(comment)):
    res = model(bert([comment.iloc[i,3]]))
    for t in res:
        if t[0]>t[1]:
            comment.iloc[i,4] = 0
        else:
            comment.iloc[i,4] = 1

100%|█████████████████████████████████████████████| 100/100 [00:44<00:00,  2.25it/s]


In [24]:
comment.to_excel("100条数据测试情感.xls")